### Scraping Pipeline for Cointerminal 

### Terms & Conditions 

When reading the terms and conditions of a website, you can search for following keywords to find restrictions:<br>
• scraper/scraping <br>
• crawler/crawling <br>
• bot <br>
• spider <br>
• program <br>

If you have no luck, you need to read through the whole legal t & c

In [1]:
import sys

#### Reverse Engineer
Detect the technology used by a website, such as Apache, JQuery, and Wordpress. Here is some example usage:


>>> builtwith('http://wordpress.com')
{u'blogs': [u'PHP', u'WordPress'], u'font-scripts': [u'Google Font API'], u'web-servers': [u'Nginx'], u'javascript-frameworks': [u'Modernizr'], u'programming-languages': [u'PHP'], u'cms': [u'WordPress']} <br>
>>> builtwith('http://webscraping.com')
{u'javascript-frameworks': [u'jQuery', u'Modernizr'], u'web-frameworks': [u'Twitter Bootstrap'], u'web-servers': [u'Nginx']} <br>


In [44]:
!pip3 install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
You should consider upgrading via the '/Users/dark_prince/.pyenv/versions/3.8.5/bin/python3.8 -m pip install --upgrade pip' command.


In [63]:
# imports for the scrapers

from builtwith import *
from urllib import robotparser
from bs4 import BeautifulSoup
from bs4 import Comment
from io import StringIO
import csv
from selenium import webdriver

import argparse
import requests
import logging
import http.client
import re
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

In [6]:
# example of what tools inside the browser does decrypt.co uses
builtwith('https://decrypt.co')

{'cdn': ['CloudFlare'],
 'javascript-graphics': ['Exhibit'],
 'font-scripts': ['Google Font API'],
 'tag-managers': ['Google Tag Manager'],
 'ecommerce': ['Intershop'],
 'javascript-frameworks': ['RequireJS', 'Wink', 'basket.js'],
 'mobile-frameworks': ['Wink']}

In [10]:
robot_parser = robotparser.RobotFileParser()


def prepare(robots_txt_url):
    robot_parser.set_url(robots_txt_url)
    robot_parser.read()


def is_allowed(target_url, user_agent='*'):
    return robot_parser.can_fetch(user_agent, target_url)


if __name__ == '__main__':
    prepare('http://www.apress.com/robots.txt')

    print(is_allowed('http://www.apress.com/covers/'))
    print(is_allowed('http://www.apress.com/gp/python'))
    


False
True


### Creating a Link Extractor

In [19]:
from urllib.request import urlopen, urljoin
import re
import requests

In [ ]:

def download_page(url):
    return urlopen(url).read().decode('utf-8')


def extract_links(page):
    link_regex = re.compile('<a[^>]+href=["\'](.*?)["\']', re.IGNORECASE)
    return link_regex.findall(page)


if __name__ == '__main__':
    target_url = 'http://www.apress.com/'
    apress = download_page(target_url)
    links = extract_links(apress)

    for link in links:
        print(urljoin(target_url, link))

### Extracting Images


In [ ]:
def download_page(url):
    return urlopen(url).read().decode('utf-8')


def extract_image_locations(page):
    img_regex = re.compile('<img[^>]+src=["\'](.*?)["\']', re.IGNORECASE)
    return img_regex.findall(page)


if __name__ == '__main__':
    target_url = 'http://www.apress.com/'
    apress = download_page(target_url)
    image_locations = extract_image_locations(apress)

    for src in image_locations:
        print(urljoin(target_url, src))

In [26]:
soup = BeautifulSoup('http://hajba.hu', 'html.parser')

/Users/dark_prince/Desktop/DissoCodes/venv/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://hajba.hu" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


To convert remote HTML pages into a soup, you should use the requests library.

In [29]:
soup = BeautifulSoup(requests.get('http://decrypt.co').text,
'html.parser')

In [46]:
soup

<!DOCTYPE html>
<html dir="ltr" lang="en-US"><head><meta content="minimum-scale=1, initial-scale=1, width=device-width, shrink-to-fit=no" name="viewport"/><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title>404 Page Not Found - Decrypt</title><meta content="The page you’re looking for is not available." name="description"/><meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/><meta content="#000" name="theme-color"/><meta content="DD213A13BA0ECDFE934884391A9E8039" name="msvalidate.01"/><meta content="2105710919682357" property="fb:app_id"/><meta content="https://www.facebook.com/decryptmedia" property="article:publisher"/><meta content="Decrypt" property="og:site_name"/><meta content="en_US" property="og:locale"/><meta content="https://decrypt.co/%20Bitcoin%20NFTs%20Are%20Emerging%20on%20Stacks.%20Can%20They%20Thrive?.html=" property="og:url"/><meta content="404 Page Not Found - Decrypt" property="og:title"/

In [51]:
url  = 'http://decrypt.co'

In [53]:
response = requests.get(url)


response

<Response [200]>

In [54]:
response.request.headers

{'User-Agent': 'python-requests/2.26.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [55]:
response.headers

{'Date': 'Fri, 08 Oct 2021 14:35:13 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Powered-By': 'Next.js', 'Content-Language': 'en-US', 'Vary': 'Origin, Accept-Encoding', 'Access-Control-Allow-Credentials': 'true', 'Cache-Control': 'public,max-age=60,s-maxage=60', 'X-Varnish': '469079084 470262607', 'Age': '6', 'Via': '1.1 varnish (Varnish/6.5)', 'x-cache': 'HIT', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v3?s=jakE7xOIWBQwmSbMFb2W2aiw%2BBYs0LDwqq9sxYOIDukFKT2g70gEFH50mGku2byWogHaUDiDdWJQNDAO2kIqDMByTISuZXBvF4zy0CI%2F%2F3SSFH7PdHXOUJ1BARA%3D"}],"group":"cf-nel","max_age":604800}', 'NEL': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'X-Content-Type-Options': '

In [57]:
# Retriving the target page text html

page = BeautifulSoup(response.text, 'html.parser')

page

<!DOCTYPE html>
<html dir="ltr" lang="en-US"><head><meta content="minimum-scale=1, initial-scale=1, width=device-width, shrink-to-fit=no" name="viewport"/><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title>Your guide to Bitcoin, Ethereum &amp; Web 3.0 - Decrypt</title><meta content="Read the latest Bitcoin and Ethereum news from Decrypt. Get the latest on cryptocurrency prices, breaking news, and more about Bitcoin and blockchain." name="description"/><meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/><meta content="#000" name="theme-color"/><meta content="DD213A13BA0ECDFE934884391A9E8039" name="msvalidate.01"/><link href="https://decrypt.co/" rel="canonical"/><meta content="2105710919682357" property="fb:app_id"/><meta content="https://www.facebook.com/decryptmedia" property="article:publisher"/><meta content="Decrypt" property="og:site_name"/><meta content="en_US" property="og:locale"/><meta content="h

In [58]:
#Obtain the title page


#page.title.string
page.title

<title>Your guide to Bitcoin, Ethereum &amp; Web 3.0 - Decrypt</title>

In [59]:
#Find all the h3 elements in the page, to determine the existing sections:

page.find_all('h3')

[<h3 class="sc-1t633a1-6 hDACPY">Insights into the day’s biggest news.</h3>,
 <h3 class="sc-1t633a1-6 hDACPY">Quickly understand the technologies that matter.</h3>,
 <h3 class="sc-1t633a1-6 hDACPY">Listen to the Decrypt Daily Podcast.</h3>,
 <h3 class="sc-1t633a1-6 hDACPY">Deep dives into Web 3.0</h3>,
 <h3 class="sc-1t633a1-6 hDACPY">The big ideas and emerging trends Decrypt cares about.</h3>,
 <h3 class="sc-1t633a1-6 hDACPY">Editorial reviews and ratings for blockchain and cryptocurrency products.</h3>,
 <h3 class="sc-1t633a1-6 hDACPY">More insights into recent blockchain news.</h3>]

In [66]:
links = page.find_all('a', href=True)
for link in links:
    print(link['href'])

/?
/search
https://twitter.com/decryptmedia
https://www.facebook.com/decryptmedia
https://www.instagram.com/decryptmedia/
https://decrypt.community
https://flipboard.com/@DecryptMedia
https://www.linkedin.com/company/decrypt-media
https://t.me/DecryptNews
https://decrypt.co/feed
https://nomics.com/
/82930/crypto-companies-raise-record-breaking-6-5b-q3-2021
/82924/figment-ceo-value-staking-is-built-community-not-one-way-street
/82920/crypto-poses-limited-risk-uk-bank-england
/82910/yearn-finance-signals-multi-chain-entrance-launch-fantom-network
/82907/us-senator-cynthia-lummis-bought-100000-bitcoin-august
/82891/bitcoin-nfts-stacks-blockchain
/resources/sorare-beginners-guide
/resources/how-to-use-coinlist-the-one-stop-shop-for-early-adopters-in-crypto
/resources/what-is-bored-ape-yacht-club-the-celebrity-nft-of-choice
/resources/what-is-ethereum-2-0
/resources/sorare-beginners-guide
/resources/how-to-use-coinlist-the-one-stop-shop-for-early-adopters-in-crypto
/resources/what-is-bored-

In [65]:
DEFAULT_PHRASE = 'python'


def process_link(source_link, text):
    logging.info(f'Extracting links from {source_link}')
    parsed_source = urlparse(source_link)
    result = requests.get(source_link)
    if result.status_code != http.client.OK:
        logging.error(f'Error retrieving {source_link}: {result}')
        return []

    if 'html' not in result.headers['Content-type']:
        logging.info(f'Link {source_link} is not an HTML page')
        return []

    page = BeautifulSoup(result.text, 'html.parser')
    search_text(source_link, page, text)

    return get_links(parsed_source, page)


def get_links(parsed_source, page):
    '''Retrieve the links on the page'''
    links = []
    for element in page.find_all('a'):
        link = element.get('href')
        if not link:
            continue

        # Avoid internal, same page links
        if link.startswith('#'):
            continue

        if link.startswith('mailto:'):
            # Ignore other links like mailto
            # More cases like ftp or similar may be included here
            continue

        # Always accept local links
        if not link.startswith('http'):
            netloc = parsed_source.netloc
            scheme = parsed_source.scheme
            path = urljoin(parsed_source.path, link)
            link = f'{scheme}://{netloc}{path}'

        # Only parse links in the same domain
        if parsed_source.netloc not in link:
            continue

        links.append(link)

    return linksDEFAULT_PHRASE = 'python'


def process_link(source_link, text):
    logging.info(f'Extracting links from {source_link}')
    parsed_source = urlparse(source_link)
    result = requests.get(source_link)
    if result.status_code != http.client.OK:
        logging.error(f'Error retrieving {source_link}: {result}')
        return []

    if 'html' not in result.headers['Content-type']:
        logging.info(f'Link {source_link} is not an HTML page')
        return []

    page = BeautifulSoup(result.text, 'html.parser')
    search_text(source_link, page, text)

    return get_links(parsed_source, page)


def get_links(parsed_source, page):
    '''Retrieve the links on the page'''
    links = []
    for element in page.find_all('a'):
        link = element.get('href')
        if not link:
            continue

        # Avoid internal, same page links
        if link.startswith('#'):
            continue

        if link.startswith('mailto:'):
            # Ignore other links like mailto
            # More cases like ftp or similar may be included here
            continue

        # Always accept local links
        if not link.startswith('http'):
            netloc = parsed_source.netloc
            scheme = parsed_source.scheme
            path = urljoin(parsed_source.path, link)
            link = f'{scheme}://{netloc}{path}'

        # Only parse links in the same domain
        if parsed_source.netloc not in link:
            continue

        links.append(link)

    return links


def search_text(source_link, page, text):
    '''Search for an element with the searched text and print it'''
    for element in page.find_all(text=re.compile(text, flags=re.IGNORECASE)):
        print(f'Link {source_link}: --> {element}')


def main(base_url, to_search):
    checked_links = set()
    to_check = [base_url]
    max_checks = 10

    while to_check and max_checks:
        link = to_check.pop(0)
        links = process_link(link, text=to_search)
        checked_links.add(link)
        for link in links:
            if link not in checked_links:
                checked_links.add(link)
                to_check.append(link)

        max_checks -= 1


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(dest='url', type=str,
                        help='Base site url. '
                              'Use  "http://localhost:8000/" '
                              'for the recipe example')
    parser.add_argument('-p', type=str,
                        help=f'Sentence to search, default: {DEFAULT_PHRASE}',
                        default=DEFAULT_PHRASE)
    args = parser.parse_args()

    main(args.url, args.p)


def search_text(source_link, page, text):
    '''Search for an element with the searched text and print it'''
    for element in page.find_all(text=re.compile(text, flags=re.IGNORECASE)):
        print(f'Link {source_link}: --> {element}')


def main(base_url, to_search):
    checked_links = set()
    to_check = [base_url]
    max_checks = 10

    while to_check and max_checks:
        link = to_check.pop(0)
        links = process_link(link, text=to_search)
        checked_links.add(link)
        for link in links:
            if link not in checked_links:
                checked_links.add(link)
                to_check.append(link)

        max_checks -= 1


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(dest='url', type=str,
                        help='Base site url. '
                              'Use  "http://localhost:8000/" '
                              'for the recipe example')
    parser.add_argument('-p', type=str,
                        help=f'Sentence to search, default: {DEFAULT_PHRASE}',
                        default=DEFAULT_PHRASE)
    args = parser.parse_args()

    main(args.url, args.p)

SyntaxError: invalid syntax (2862921048.py, line 52)